## Выполнили:
<br> Соболева Елизавета
<br> Гаспарян Генрих
<br> Плямоватый Даниил
<br> БММ 181

In [88]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool

In [77]:
data = pd.read_csv('./ratings.csv')
data

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


Используем одну из SotA методик - разложение матрицы оценок с помощью SVD и последущее применение catboost.

In [78]:
data_train, data_test = train_test_split(data, test_size=0.3)

## Получим вектора, кодирующие пользователей и фильмы

In [79]:
matrix_train = data_train.pivot(index='userId', columns='movieId', values='rating')
matrix_train

movieId,1,2,3,4,5,6,7,8,9,10,...,190221,191005,193567,193571,193573,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
# Чтобы учесть баес, связанный с конкретным пользователем, найдем его средние оценки. 
matrix = matrix_train.values
user_ratings_mean = np.mean(matrix, axis = 1)
matrix_norm = matrix - user_ratings_mean.reshape(-1, 1)

In [81]:
matrix_train.shape

(610, 8594)

Используем факторизацию матрицы

In [82]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(matrix_norm, k = 50)

/home/runfme/anaconda3/lib/python3.7/site-packages/scipy/sparse/linalg/eigen/arpack/arpack.py:1883: RuntimeWarning: invalid value encountered in greater
  above_cutoff = (eigvals > cutoff)


Теперь в U лежат значения векторов для пользователей, а Vt для фильмов.

## catboost

Теперь создадим наш датасет и применим catboost.

In [83]:
data_train['userId'].apply(user_feat_getter)

33673    [0.002577507806823682, -0.01454379816868426, 0...
28130    [0.007683347354050377, 0.02473714036472693, -0...
14238    [-0.01728253657212014, 0.006790707887859617, 0...
91007    [-0.008975754041467995, -0.02452971931292662, ...
15145    [-0.0019737430419386093, -0.001957050860954374...
                               ...                        
8806     [-0.0064102565941943565, -0.014069207393965952...
67863    [0.08797068117375179, 0.059643988241803655, 0....
64926    [-0.05821515603496566, -0.009706133274821354, ...
92922    [-0.0228331421111311, -0.016619593946997843, 0...
99865    [-0.022999283915677876, -0.007469628202602213,...
Name: userId, Length: 70585, dtype: object

In [84]:
def user_feat_getter(userid):
    if userid in matrix_train.index:
        return U[matrix_train.index.get_loc(userid)]
    else:
        print(f"didn't find user {userid}")
        return np.zeros(50)
def movie_feat_getter(movieid):
    if movieid in matrix_train.columns:
        return Vt[:, matrix_train.columns.get_loc(movieid)]
    else:
        print(f"didn't find movie {movieid}")
        return np.zeros(50)
    
data_train['user_feats'] = data['userId'].apply(user_feat_getter)
data_train['movie_feats'] = data['movieId'].apply(movie_feat_getter)
data_test['user_feats'] = data['userId'].apply(user_feat_getter)
data_test['movie_feats'] = data['movieId'].apply(movie_feat_getter)

/home/runfme/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


didn't find movie 4252
didn't find movie 4260
didn't find movie 201
didn't find movie 243
didn't find movie 254
didn't find movie 310
didn't find movie 510
didn't find movie 747
didn't find movie 6044
didn't find movie 77841
didn't find movie 105211
didn't find movie 137595
didn't find movie 27716
didn't find movie 51357
didn't find movie 88094
didn't find movie 141688
didn't find movie 144210
didn't find movie 155168
didn't find movie 295
didn't find movie 1798
didn't find movie 1891
didn't find movie 2037
didn't find movie 2041
didn't find movie 2614
didn't find movie 2898
didn't find movie 2900
didn't find movie 6143
didn't find movie 52730
didn't find movie 59429
didn't find movie 73858
didn't find movie 85259
didn't find movie 103801
didn't find movie 105653
didn't find movie 173307
didn't find movie 68536
didn't find movie 4251
didn't find movie 6997
didn't find movie 7884
didn't find movie 8378
didn't find movie 26901
didn't find movie 27450
didn't find movie 49265
didn't find m

didn't find movie 4521
didn't find movie 4704
didn't find movie 4860
didn't find movie 5168
didn't find movie 5213
didn't find movie 5384
didn't find movie 5455
didn't find movie 5601
didn't find movie 5602
didn't find movie 5603
didn't find movie 5604
didn't find movie 5633
didn't find movie 5649
didn't find movie 5771
didn't find movie 5875
didn't find movie 6101
didn't find movie 6163
didn't find movie 6245
didn't find movie 6315
didn't find movie 6336
didn't find movie 6454
didn't find movie 6527
didn't find movie 6581
didn't find movie 6603
didn't find movie 6660
didn't find movie 6768
didn't find movie 6837
didn't find movie 6945
didn't find movie 6948
didn't find movie 6962
didn't find movie 6965
didn't find movie 7012
didn't find movie 7057
didn't find movie 7058
didn't find movie 7062
didn't find movie 7083
didn't find movie 7092
didn't find movie 7167
didn't find movie 7177
didn't find movie 7305
didn't find movie 7414
didn't find movie 7560
didn't find movie 7647
didn't find

/home/runfme/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/runfme/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


didn't find movie 4252
didn't find movie 4260
didn't find movie 201
didn't find movie 243
didn't find movie 254
didn't find movie 310
didn't find movie 510
didn't find movie 747
didn't find movie 6044
didn't find movie 77841
didn't find movie 105211
didn't find movie 137595
didn't find movie 27716
didn't find movie 51357
didn't find movie 88094
didn't find movie 141688
didn't find movie 144210
didn't find movie 155168
didn't find movie 295
didn't find movie 1798
didn't find movie 1891
didn't find movie 2037
didn't find movie 2041
didn't find movie 2614
didn't find movie 2898
didn't find movie 2900
didn't find movie 6143
didn't find movie 52730
didn't find movie 59429
didn't find movie 73858
didn't find movie 85259
didn't find movie 103801
didn't find movie 105653
didn't find movie 173307
didn't find movie 68536
didn't find movie 4251
didn't find movie 6997
didn't find movie 7884
didn't find movie 8378
didn't find movie 26901
didn't find movie 27450
didn't find movie 49265
didn't find m

didn't find movie 5168
didn't find movie 5213
didn't find movie 5384
didn't find movie 5455
didn't find movie 5601
didn't find movie 5602
didn't find movie 5603
didn't find movie 5604
didn't find movie 5633
didn't find movie 5649
didn't find movie 5771
didn't find movie 5875
didn't find movie 6101
didn't find movie 6163
didn't find movie 6245
didn't find movie 6315
didn't find movie 6336
didn't find movie 6454
didn't find movie 6527
didn't find movie 6581
didn't find movie 6603
didn't find movie 6660
didn't find movie 6768
didn't find movie 6837
didn't find movie 6945
didn't find movie 6948
didn't find movie 6962
didn't find movie 6965
didn't find movie 7012
didn't find movie 7057
didn't find movie 7058
didn't find movie 7062
didn't find movie 7083
didn't find movie 7092
didn't find movie 7167
didn't find movie 7177
didn't find movie 7305
didn't find movie 7414
didn't find movie 7560
didn't find movie 7647
didn't find movie 7702
didn't find movie 7839
didn't find movie 7900
didn't find

/home/runfme/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [92]:
X_train = np.concatenate([
    np.array(list(data_train['user_feats'])), 
    np.array(list(data_train['movie_feats']))], 
    axis=1)
X_test = np.concatenate([
    np.array(list(data_test['user_feats'])), 
    np.array(list(data_test['movie_feats']))], 
    axis=1)

y_train = data_train['rating']
y_test = data_test['rating']

In [93]:
X_train.shape

(70585, 100)

In [97]:
cat = CatBoostRegressor(eval_metric='RMSE')

test_set = Pool(X_test, y_test)
cat.fit(X_train, y_train, eval_set=test_set)

Learning rate set to 0.103013
0:	learn: 1.0354670	test: 1.0324695	best: 1.0324695 (0)	total: 59.7ms	remaining: 59.7s
1:	learn: 1.0292921	test: 1.0265867	best: 1.0265867 (1)	total: 82.3ms	remaining: 41.1s
2:	learn: 1.0237244	test: 1.0210888	best: 1.0210888 (2)	total: 102ms	remaining: 34s
3:	learn: 1.0190700	test: 1.0164797	best: 1.0164797 (3)	total: 122ms	remaining: 30.3s
4:	learn: 1.0157566	test: 1.0135262	best: 1.0135262 (4)	total: 140ms	remaining: 27.9s
5:	learn: 1.0117350	test: 1.0095913	best: 1.0095913 (5)	total: 160ms	remaining: 26.5s
6:	learn: 1.0078751	test: 1.0056791	best: 1.0056791 (6)	total: 180ms	remaining: 25.5s
7:	learn: 1.0043274	test: 1.0024525	best: 1.0024525 (7)	total: 206ms	remaining: 25.5s
8:	learn: 1.0019943	test: 1.0002906	best: 1.0002906 (8)	total: 230ms	remaining: 25.4s
9:	learn: 0.9983896	test: 0.9968731	best: 0.9968731 (9)	total: 250ms	remaining: 24.8s
10:	learn: 0.9948193	test: 0.9935613	best: 0.9935613 (10)	total: 272ms	remaining: 24.4s
11:	learn: 0.9921044	t

94:	learn: 0.9256354	test: 0.9342661	best: 0.9342661 (94)	total: 2.14s	remaining: 20.4s
95:	learn: 0.9251400	test: 0.9338747	best: 0.9338747 (95)	total: 2.16s	remaining: 20.3s
96:	learn: 0.9246548	test: 0.9334955	best: 0.9334955 (96)	total: 2.18s	remaining: 20.3s
97:	learn: 0.9239460	test: 0.9329708	best: 0.9329708 (97)	total: 2.2s	remaining: 20.2s
98:	learn: 0.9234887	test: 0.9326467	best: 0.9326467 (98)	total: 2.22s	remaining: 20.2s
99:	learn: 0.9230717	test: 0.9323373	best: 0.9323373 (99)	total: 2.24s	remaining: 20.1s
100:	learn: 0.9223973	test: 0.9318335	best: 0.9318335 (100)	total: 2.26s	remaining: 20.1s
101:	learn: 0.9218411	test: 0.9313663	best: 0.9313663 (101)	total: 2.28s	remaining: 20.1s
102:	learn: 0.9212604	test: 0.9309257	best: 0.9309257 (102)	total: 2.3s	remaining: 20s
103:	learn: 0.9206170	test: 0.9304879	best: 0.9304879 (103)	total: 2.32s	remaining: 20s
104:	learn: 0.9200828	test: 0.9301347	best: 0.9301347 (104)	total: 2.34s	remaining: 20s
105:	learn: 0.9195967	test: 0.

196:	learn: 0.8875838	test: 0.9091403	best: 0.9091403 (196)	total: 4.31s	remaining: 17.6s
197:	learn: 0.8873550	test: 0.9090350	best: 0.9090350 (197)	total: 4.33s	remaining: 17.5s
198:	learn: 0.8869899	test: 0.9088334	best: 0.9088334 (198)	total: 4.35s	remaining: 17.5s
199:	learn: 0.8867102	test: 0.9086682	best: 0.9086682 (199)	total: 4.36s	remaining: 17.5s
200:	learn: 0.8864499	test: 0.9085201	best: 0.9085201 (200)	total: 4.38s	remaining: 17.4s
201:	learn: 0.8860955	test: 0.9082705	best: 0.9082705 (201)	total: 4.4s	remaining: 17.4s
202:	learn: 0.8857852	test: 0.9080534	best: 0.9080534 (202)	total: 4.42s	remaining: 17.4s
203:	learn: 0.8856076	test: 0.9079719	best: 0.9079719 (203)	total: 4.44s	remaining: 17.3s
204:	learn: 0.8854367	test: 0.9078973	best: 0.9078973 (204)	total: 4.46s	remaining: 17.3s
205:	learn: 0.8852801	test: 0.9078405	best: 0.9078405 (205)	total: 4.47s	remaining: 17.3s
206:	learn: 0.8850655	test: 0.9077107	best: 0.9077107 (206)	total: 4.5s	remaining: 17.2s
207:	learn: 

294:	learn: 0.8661253	test: 0.8995078	best: 0.8995078 (294)	total: 6.24s	remaining: 14.9s
295:	learn: 0.8658909	test: 0.8993902	best: 0.8993902 (295)	total: 6.26s	remaining: 14.9s
296:	learn: 0.8657297	test: 0.8992871	best: 0.8992871 (296)	total: 6.28s	remaining: 14.9s
297:	learn: 0.8655324	test: 0.8990940	best: 0.8990940 (297)	total: 6.3s	remaining: 14.8s
298:	learn: 0.8653539	test: 0.8990450	best: 0.8990450 (298)	total: 6.32s	remaining: 14.8s
299:	learn: 0.8651506	test: 0.8989833	best: 0.8989833 (299)	total: 6.33s	remaining: 14.8s
300:	learn: 0.8650024	test: 0.8989506	best: 0.8989506 (300)	total: 6.35s	remaining: 14.8s
301:	learn: 0.8648191	test: 0.8989072	best: 0.8989072 (301)	total: 6.37s	remaining: 14.7s
302:	learn: 0.8646409	test: 0.8987777	best: 0.8987777 (302)	total: 6.39s	remaining: 14.7s
303:	learn: 0.8644975	test: 0.8987723	best: 0.8987723 (303)	total: 6.41s	remaining: 14.7s
304:	learn: 0.8643322	test: 0.8987339	best: 0.8987339 (304)	total: 6.43s	remaining: 14.6s
305:	learn:

392:	learn: 0.8498924	test: 0.8951508	best: 0.8951392 (391)	total: 8.15s	remaining: 12.6s
393:	learn: 0.8497291	test: 0.8950866	best: 0.8950866 (393)	total: 8.17s	remaining: 12.6s
394:	learn: 0.8495840	test: 0.8950600	best: 0.8950600 (394)	total: 8.19s	remaining: 12.5s
395:	learn: 0.8494128	test: 0.8950007	best: 0.8950007 (395)	total: 8.21s	remaining: 12.5s
396:	learn: 0.8493004	test: 0.8949479	best: 0.8949479 (396)	total: 8.23s	remaining: 12.5s
397:	learn: 0.8491071	test: 0.8949463	best: 0.8949463 (397)	total: 8.25s	remaining: 12.5s
398:	learn: 0.8489469	test: 0.8949284	best: 0.8949284 (398)	total: 8.27s	remaining: 12.5s
399:	learn: 0.8487785	test: 0.8948629	best: 0.8948629 (399)	total: 8.29s	remaining: 12.4s
400:	learn: 0.8486491	test: 0.8948575	best: 0.8948575 (400)	total: 8.31s	remaining: 12.4s
401:	learn: 0.8484442	test: 0.8948522	best: 0.8948522 (401)	total: 8.32s	remaining: 12.4s
402:	learn: 0.8482884	test: 0.8947891	best: 0.8947891 (402)	total: 8.34s	remaining: 12.4s
403:	learn

...

594:	learn: 0.8200928	test: 0.8908232	best: 0.8908232 (594)	total: 12.7s	remaining: 8.65s
595:	learn: 0.8200092	test: 0.8908110	best: 0.8908110 (595)	total: 12.7s	remaining: 8.63s
596:	learn: 0.8198375	test: 0.8907759	best: 0.8907759 (596)	total: 12.7s	remaining: 8.6s
597:	learn: 0.8196852	test: 0.8907982	best: 0.8907759 (596)	total: 12.8s	remaining: 8.58s
598:	learn: 0.8195771	test: 0.8907778	best: 0.8907759 (596)	total: 12.8s	remaining: 8.56s
599:	learn: 0.8194679	test: 0.8907551	best: 0.8907551 (599)	total: 12.8s	remaining: 8.54s
600:	learn: 0.8193352	test: 0.8907394	best: 0.8907394 (600)	total: 12.8s	remaining: 8.52s
601:	learn: 0.8192069	test: 0.8907089	best: 0.8907089 (601)	total: 12.8s	remaining: 8.5s
602:	learn: 0.8190860	test: 0.8907213	best: 0.8907089 (601)	total: 12.9s	remaining: 8.47s
603:	learn: 0.8189636	test: 0.8907229	best: 0.8907089 (601)	total: 12.9s	remaining: 8.45s
604:	learn: 0.8188201	test: 0.8907371	best: 0.8907089 (601)	total: 12.9s	remaining: 8.43s
605:	learn: 

692:	learn: 0.8073670	test: 0.8901735	best: 0.8901271 (679)	total: 14.9s	remaining: 6.6s
693:	learn: 0.8072527	test: 0.8901910	best: 0.8901271 (679)	total: 14.9s	remaining: 6.58s
694:	learn: 0.8071386	test: 0.8901887	best: 0.8901271 (679)	total: 14.9s	remaining: 6.55s
695:	learn: 0.8070039	test: 0.8902068	best: 0.8901271 (679)	total: 15s	remaining: 6.53s
696:	learn: 0.8069016	test: 0.8901786	best: 0.8901271 (679)	total: 15s	remaining: 6.51s
697:	learn: 0.8067632	test: 0.8902120	best: 0.8901271 (679)	total: 15s	remaining: 6.49s
698:	learn: 0.8066751	test: 0.8902200	best: 0.8901271 (679)	total: 15s	remaining: 6.46s
699:	learn: 0.8065502	test: 0.8901996	best: 0.8901271 (679)	total: 15s	remaining: 6.44s
700:	learn: 0.8064610	test: 0.8902025	best: 0.8901271 (679)	total: 15s	remaining: 6.42s
701:	learn: 0.8063276	test: 0.8901841	best: 0.8901271 (679)	total: 15.1s	remaining: 6.39s
702:	learn: 0.8062325	test: 0.8901885	best: 0.8901271 (679)	total: 15.1s	remaining: 6.37s
703:	learn: 0.8060966	t

789:	learn: 0.7952069	test: 0.8896417	best: 0.8896148 (788)	total: 17s	remaining: 4.53s
790:	learn: 0.7950643	test: 0.8896456	best: 0.8896148 (788)	total: 17.1s	remaining: 4.51s
791:	learn: 0.7949086	test: 0.8896857	best: 0.8896148 (788)	total: 17.1s	remaining: 4.49s
792:	learn: 0.7947851	test: 0.8896966	best: 0.8896148 (788)	total: 17.1s	remaining: 4.46s
793:	learn: 0.7946655	test: 0.8897399	best: 0.8896148 (788)	total: 17.1s	remaining: 4.44s
794:	learn: 0.7945162	test: 0.8897244	best: 0.8896148 (788)	total: 17.1s	remaining: 4.42s
795:	learn: 0.7944073	test: 0.8897102	best: 0.8896148 (788)	total: 17.2s	remaining: 4.4s
796:	learn: 0.7942996	test: 0.8896750	best: 0.8896148 (788)	total: 17.2s	remaining: 4.38s
797:	learn: 0.7941797	test: 0.8896682	best: 0.8896148 (788)	total: 17.2s	remaining: 4.35s
798:	learn: 0.7941120	test: 0.8896661	best: 0.8896148 (788)	total: 17.2s	remaining: 4.33s
799:	learn: 0.7940005	test: 0.8896469	best: 0.8896148 (788)	total: 17.2s	remaining: 4.31s
800:	learn: 0

887:	learn: 0.7832841	test: 0.8892992	best: 0.8892505 (879)	total: 19.2s	remaining: 2.43s
888:	learn: 0.7831657	test: 0.8893012	best: 0.8892505 (879)	total: 19.3s	remaining: 2.4s
889:	learn: 0.7830390	test: 0.8893047	best: 0.8892505 (879)	total: 19.3s	remaining: 2.38s
890:	learn: 0.7829222	test: 0.8892944	best: 0.8892505 (879)	total: 19.3s	remaining: 2.36s
891:	learn: 0.7828280	test: 0.8892797	best: 0.8892505 (879)	total: 19.3s	remaining: 2.34s
892:	learn: 0.7827036	test: 0.8893079	best: 0.8892505 (879)	total: 19.3s	remaining: 2.32s
893:	learn: 0.7825933	test: 0.8893138	best: 0.8892505 (879)	total: 19.4s	remaining: 2.29s
894:	learn: 0.7825150	test: 0.8892896	best: 0.8892505 (879)	total: 19.4s	remaining: 2.27s
895:	learn: 0.7823733	test: 0.8892957	best: 0.8892505 (879)	total: 19.4s	remaining: 2.25s
896:	learn: 0.7822614	test: 0.8893423	best: 0.8892505 (879)	total: 19.4s	remaining: 2.23s
897:	learn: 0.7821379	test: 0.8893471	best: 0.8892505 (879)	total: 19.4s	remaining: 2.21s
898:	learn:

984:	learn: 0.7722334	test: 0.8889844	best: 0.8889844 (984)	total: 21.4s	remaining: 326ms
985:	learn: 0.7721377	test: 0.8890027	best: 0.8889844 (984)	total: 21.4s	remaining: 305ms
986:	learn: 0.7720140	test: 0.8889938	best: 0.8889844 (984)	total: 21.5s	remaining: 283ms
987:	learn: 0.7718824	test: 0.8889911	best: 0.8889844 (984)	total: 21.5s	remaining: 261ms
988:	learn: 0.7717863	test: 0.8889999	best: 0.8889844 (984)	total: 21.5s	remaining: 239ms
989:	learn: 0.7716671	test: 0.8889744	best: 0.8889744 (989)	total: 21.5s	remaining: 217ms
990:	learn: 0.7715834	test: 0.8889807	best: 0.8889744 (989)	total: 21.5s	remaining: 196ms
991:	learn: 0.7715003	test: 0.8889781	best: 0.8889744 (989)	total: 21.6s	remaining: 174ms
992:	learn: 0.7714013	test: 0.8889875	best: 0.8889744 (989)	total: 21.6s	remaining: 152ms
993:	learn: 0.7712438	test: 0.8889947	best: 0.8889744 (989)	total: 21.6s	remaining: 130ms
994:	learn: 0.7711138	test: 0.8889806	best: 0.8889744 (989)	total: 21.6s	remaining: 109ms
995:	learn